In [8]:
from fyers_api import fyersModel
from fyers_api import accessToken
import pandas as pd
import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv("../.env")


True

In [10]:
client_id=os.environ.get('API_CLIENT_ID')
secret_key=os.environ.get('API_SECRET_KEY')
redirect_uri=os.environ.get('API_REDIRECT_URL')


In [11]:
session=accessToken.SessionModel(client_id=client_id,
secret_key=secret_key,redirect_uri=redirect_uri, 
response_type='code', grant_type='authorization_code')

In [12]:
auth_code_provider = session.generate_authcode()
print(auth_code_provider)

https://api.fyers.in/api/v2/generate-authcode?client_id=QW4JS7GAG2-100&redirect_uri=https%3A%2F%2F127.0.0.1%3A5000%2Flogin&response_type=code&state=None


In [13]:
auth_code=str(input("Enter the auth_code: "))

In [14]:
session.set_token(auth_code)
response = session.generate_token()
access_token = response["access_token"]
if(access_token is None):
    print("Access token is not generated")
else:
    print("Access token is generated")
    

Access token is generated


In [15]:
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)
is_async = True

In [16]:
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'XA15265',
  'name': 'AIYAZ ANWAR QURESHI',
  'image': None,
  'display_name': None,
  'pin_change_date': '21-12-2021 14:00:15',
  'email_id': 'qureshi.aiyaz123@gmail.com',
  'pwd_change_date': '18-08-2021 13:45:28;41',
  'PAN': '---------',
  'pwd_to_expire': 90}}

In [17]:
# Getting Data
def dynamicDataFetching(startDate,endDate,symbol):
    # Creating a dataframe to store the data
    df = pd.DataFrame(columns=["date","open","high","low","close","volume"])
    today = dt.date.today()
    range_from=startDate
    while(range_from<=today):
    # Check whether the upcoming endDate is lesser than today if we add 99D or else make it today
        if((range_from + dt.timedelta(days=99))<today):
            range_to = range_from + dt.timedelta(days=99)
        else:
            range_to = today

        # Api call 
        data = {"symbol":symbol,"resolution":"5","date_format":"1","range_from":range_from,"range_to":range_to,"cont_flag":"1"}
        res = fyers.history(data)
        res = res["candles"]
        
        # Converting the data into pandas dataframe 
        tempDf = pd.DataFrame(columns=["date","open","high","low","close","volume"])
        for i in range(len(res)):
             tempDf.loc[i]=[res[i][0],res[i][1],res[i][2],res[i][3],res[i][4],res[i][5]]
       
        # Appending the data to the main dataframe
        df = pd.concat([df,tempDf])

        # Updating the range_from to last date of last api call
        range_from=range_to+dt.timedelta(days=1)
    return df

In [19]:
startDate = dt.date(2021,7,1)
endDate = dt.date.today()
symbol="NSE:TATASTEEL-EQ"

df = dynamicDataFetching(startDate,endDate,symbol)
print(df.dtypes)
print(df.describe())

date      object
open      object
high      object
low       object
close     object
volume    object
dtype: object


In [20]:
# Converting everything to playable format
df["date"]= pd.to_datetime(df["date"],unit='s')
df.set_index("date",inplace=True)
df["open"]=df["open"].astype(float)
df["high"]=df["high"].astype(float)
df["low"]=df["low"].astype(float)
df["close"]=df["close"].astype(float)
df["volume"]=df["volume"].astype(float)
 
print(df.head())
print("\nData types :\n" , df.dtypes)

                       open    high     low   close     volume
date                                                          
2021-07-01 03:45:00  117.19  117.19  115.65  116.31  4809510.0
2021-07-01 03:50:00  116.35  117.20  116.26  116.97  2844120.0
2021-07-01 03:55:00  116.93  117.16  116.70  117.03  1459550.0
2021-07-01 04:00:00  117.04  117.45  116.85  117.40  1677160.0
2021-07-01 04:05:00  117.38  117.68  117.32  117.36  2206000.0

Data types :
 open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object


In [ ]:
# Export df to csv
fileName = "TataSteel.csv"
df.to_csv(fileName)